In [1]:
import os
import glob
import pandas as pd
from pandas import value_counts

# groundtruth 目录：改成你自己的路径
GROUNDTRUTH_DIR = r"C:\Users\zyf17\Desktop\NOT-IOT\SecurityEvents_Network_datasets"
PATTERN = "groundtruth_network_*.csv"

csv_files = sorted(glob.glob(os.path.join(GROUNDTRUTH_DIR, PATTERN)))
print(f"发现 {len(csv_files)} 个 groundtruth CSV:")
for f in csv_files:
    print(" -", os.path.basename(f))

dfs = []
for f in csv_files:
    df_part = pd.read_csv(f)
    df_part["source_file"] = os.path.basename(f)
    dfs.append(df_part)

df = pd.concat(dfs, ignore_index=True)
print("合并后总行数:", len(df))
print("列名:", df.columns.tolist())

# ts 转成可比较的数值（不转 datetime，只保证 float）
df["ts"] = pd.to_numeric(df["ts"], errors="coerce")
before = len(df)
df = df.dropna(subset=["ts"])
df["ts"] = df["ts"].astype(float)
print("丢弃 ts 无法解析行数:", before - len(df))
print("最终行数:", len(df))


发现 18 个 groundtruth CSV:
 - GroundTruth_Network_1.csv
 - GroundTruth_Network_10.csv
 - GroundTruth_Network_11.csv
 - GroundTruth_Network_12.csv
 - GroundTruth_Network_13.csv
 - GroundTruth_Network_14.csv
 - GroundTruth_Network_15.csv
 - GroundTruth_Network_16.csv
 - GroundTruth_Network_17.csv
 - GroundTruth_Network_18.csv
 - GroundTruth_Network_2.csv
 - GroundTruth_Network_3.csv
 - GroundTruth_Network_4.csv
 - GroundTruth_Network_5.csv
 - GroundTruth_Network_6.csv
 - GroundTruth_Network_7.csv
 - GroundTruth_Network_8.csv
 - GroundTruth_Network_9.csv
合并后总行数: 17801188
列名: ['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'proto', 'type', 'source_file']
丢弃 ts 无法解析行数: 0
最终行数: 17801188


In [5]:
def normalize_5tuple_vectorized(df):
    src_ip = df["src_ip"].astype(str)
    dst_ip = df["dst_ip"].astype(str)

    # 2. Create a Boolean mask for swapping
    # The condition is: should we swap (i.e., is (src_ip, src_port) > (dst_ip, dst_port))?
    # Because string comparison on IPs can be tricky, it's safer to compare the columns explicitly.
    # The comparison logic is: (src_ip > dst_ip) OR (src_ip == dst_ip AND src_port > dst_port)

    # Cast ports to integer if they aren't already
    src_port = df["src_port"].astype(int)
    dst_port = df["dst_port"].astype(int)

    # Create the mask for rows where a swap is necessary
    swap_mask = (src_ip > dst_ip) | ((src_ip == dst_ip) & (src_port > dst_port))

    # 3. Apply the swap using the mask

    # Initialize the normalized columns with the original values
    df['ip1'] = src_ip
    df['port1'] = src_port
    df['ip2'] = dst_ip
    df['port2'] = dst_port

    # For rows where swap_mask is True:

    # ip1 gets dst_ip, ip2 gets src_ip (SWAP)
    df.loc[swap_mask, 'ip1'] = dst_ip[swap_mask]
    df.loc[swap_mask, 'ip2'] = src_ip[swap_mask]

    # port1 gets dst_port, port2 gets src_port (SWAP)
    df.loc[swap_mask, 'port1'] = dst_port[swap_mask]
    df.loc[swap_mask, 'port2'] = src_port[swap_mask]

    # The protocol column is already normalized (it's the same regardless of direction)
    df['proto'] = df["proto"].astype(str)

    return df[['ip1', 'port1', 'ip2', 'port2', 'proto']]

# --- How to use it ---

# Assuming your original DataFrame is 'df'
df_norm_cols = normalize_5tuple_vectorized(df.copy())
df_norm_cols = df_norm_cols.drop(columns=['proto'])
df_norm = df.join(df_norm_cols)
df_norm.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,type,source_file,ip1,port1,ip2,port2
0,1.556022e+09,192.168.1.31,38140,192.168.1.49,1185,tcp,scanning,GroundTruth_Network_1.csv,192.168.1.31,38140,192.168.1.49,1185
1,1.556022e+09,192.168.1.31,50945,192.168.1.186,5815,tcp,scanning,GroundTruth_Network_1.csv,192.168.1.186,5815,192.168.1.31,50945
2,1.556022e+09,192.168.1.31,50945,192.168.1.250,3011,tcp,scanning,GroundTruth_Network_1.csv,192.168.1.250,3011,192.168.1.31,50945
3,1.556022e+09,192.168.1.31,50946,192.168.1.49,32775,tcp,scanning,GroundTruth_Network_1.csv,192.168.1.31,50946,192.168.1.49,32775
4,1.556022e+09,192.168.1.31,50950,192.168.1.133,1000,tcp,scanning,GroundTruth_Network_1.csv,192.168.1.133,1000,192.168.1.31,50950


In [6]:
attack_overview = (
    df_norm.groupby("type")["ts"]
    .agg(global_min_ts="min", global_max_ts="max", count="count")
    .reset_index()
    .sort_values("type")
)
attack_overview

,type,global_min_ts,global_max_ts,count
0,backdoor,1.556437e+09,1.556531e+09,188864
1,ddos,1.556204e+09,1.556264e+09,6031012
2,dos,1.556130e+09,1.556161e+09,196227
3,injection,1.556171e+09,1.556187e+09,451136
4,mitm,1.556540e+09,1.556549e+09,1010
5,password,1.556286e+09,1.556337e+09,1253031
6,ransomware,1.556423e+09,1.556432e+09,5629
7,scanning,1.556021e+09,1.556098e+09,7138336
8,xss,1.556341e+09,1.556369e+09,2535943


In [7]:
attack_overview.to_csv("attack_overview_time_ranges.csv", index=False)
print("已保存 attack_overview_time_ranges.csv")

已保存 attack_overview_time_ranges.csv


In [8]:
five_tuple_cols = ["ip1", "port1", "ip2", "port2", "proto"]
attack_5tuple_ranges = (
    df_norm[["type", "ts"] + five_tuple_cols]
    .groupby(["type"] + five_tuple_cols)["ts"]
    .agg(min_ts="min", max_ts="max", count="count")
    .reset_index()
    .sort_values(["type"] + five_tuple_cols)
)
attack_5tuple_ranges.head()

,type,ip1,port1,ip2,port2,proto,min_ts,max_ts,count
0,backdoor,103.38.120.36,123,192.168.1.33,33156,udp,1.556461e+09,1.556461e+09,1
1,backdoor,103.38.120.36,123,192.168.1.33,33514,udp,1.556459e+09,1.556459e+09,1
2,backdoor,103.38.120.36,123,192.168.1.33,33801,udp,1.556477e+09,1.556477e+09,1
3,backdoor,103.38.120.36,123,192.168.1.33,35405,udp,1.556498e+09,1.556498e+09,1
4,backdoor,103.38.120.36,123,192.168.1.33,35680,udp,1.556522e+09,1.556522e+09,1


In [9]:
attack_5tuple_ranges.to_csv("attack_5tuple_time_ranges.csv", index=False)
print("已保存 attack_5tuple_time_ranges.csv")

已保存 attack_5tuple_time_ranges.csv


In [12]:
df = pd.read_csv(r"C:\Users\zyf17\Desktop\NOT-IOT\SecurityEvents_Network_datasets\attack_5tuple_time_ranges.csv")

df_pass = df[df["type"] == "injection"]

In [13]:
df_pass['count'].sum()

np.int64(451136)

In [16]:
import pandas as pd
import glob
import os
from tqdm import tqdm

def count_attack_types(directory_path):
    """
    统计目录下所有 CSV 文件中 'type' 列的值分布。
    只读取 'type' 列以节省内存。
    """
    # 1. 获取所有 CSV 文件路径
    # 假设你的系统是 Windows，如果路径有特殊字符记得加 r""
    csv_files = glob.glob(os.path.join(directory_path, "*.csv"))

    if not csv_files:
        print(f"❌ 错误: 在 {directory_path} 下没有找到 .csv 文件")
        return

    print(f"📂 找到 {len(csv_files)} 个文件，开始统计...")

    # 2. 初始化一个空的 Series 来存储总计数
    total_counts = pd.Series(dtype=int)

    # 3. 逐个文件处理
    for file_path in tqdm(csv_files, desc="Processing CSVs", unit="file"):
        try:
            # 核心优化：usecols=['type']
            # 只读取这一列，速度极快，内存占用极小
            df = pd.read_csv(file_path, usecols=['type'])

            # 计算当前文件的计数
            current_counts = df['type'].value_counts()

            # 将当前文件的计数加到总计数中
            # fill_value=0 保证如果新文件出现了旧文件没有的攻击类型，不会报错
            total_counts = total_counts.add(current_counts, fill_value=0)

        except ValueError:
            print(f"\n⚠️ 警告: 文件 {os.path.basename(file_path)} 中没有找到 'type' 列，已跳过。")
        except Exception as e:
            print(f"\n❌ 读取 {os.path.basename(file_path)} 失败: {e}")

    # 4. 整理结果
    # 排序：数量多的在前面
    total_counts = total_counts.sort_values(ascending=False).astype(int)

    print("\n" + "="*40)
    print("📊 最终统计结果 (Type Value Counts)")
    print("="*40)
    print(total_counts)

    # 可选：保存到文件
    # total_counts.to_csv("type_counts_result.csv")

# ================= 使用示例 =================
# 请把这里的路径换成你 CSV 所在的真实目录路径
target_dir = r"C:\Users\zyf17\Desktop\NOT-IOT\processed"

count_attack_types(target_dir)

📂 找到 23 个文件，开始统计...


Processing CSVs: 100%|██████████| 23/23 [00:09<00:00,  2.46file/s]


📊 最终统计结果 (Type Value Counts)
type
scanning      7140161
ddos          6165008
dos           3375328
xss           2108944
password      1718568
normal         796380
backdoor       508116
injection      452659
ransomware      72805
mitm             1052
dtype: int64


In [16]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm import tqdm

def suggest_truncation_thresholds(folder_path, attack_name, retention_target=0.995):
    """
    分析 Duration, Packet Count, Byte Size 分布，并建议截断阈值。
    retention_target=0.995 意味着我们希望 99.5% 的数据都不被截断。
    """
    # 1. 寻找文件
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not csv_files:
        print("❌ 未找到 CSV 文件")
        return

    print(f"📂 正在扫描 {len(csv_files)} 个文件，分析攻击类型: '{attack_name}' ...")

    # 2. 只需要读取这几列
    use_cols = ['type', 'duration', 'src_pkts', 'dst_pkts', 'src_bytes', 'dst_bytes']

    all_durations = []
    all_pkts = []
    all_bytes = []

    for f_path in tqdm(csv_files, unit="file"):
        try:
            df = pd.read_csv(f_path, usecols=lambda c: c in use_cols, low_memory=False)

            # 清洗攻击类型
            df['type_clean'] = df['type'].astype(str).str.strip().str.lower()
            target_df = df[df['type_clean'] == attack_name.lower()]

            if target_df.empty:
                continue

            # --- 提取并处理数据 ---

            # Duration
            d = pd.to_numeric(target_df['duration'], errors='coerce').fillna(0.0).values
            all_durations.extend(d)

            # Packets (Src + Dst)
            sp = pd.to_numeric(target_df['src_pkts'], errors='coerce').fillna(0).values
            dp = pd.to_numeric(target_df['dst_pkts'], errors='coerce').fillna(0).values
            all_pkts.extend(sp + dp)

            # Bytes (Src + Dst) - 用来评估 MongoDB 文档大小
            sb = pd.to_numeric(target_df['src_bytes'], errors='coerce').fillna(0).values
            db = pd.to_numeric(target_df['dst_bytes'], errors='coerce').fillna(0).values
            all_bytes.extend(sb + db)

        except Exception as e:
            print(f"⚠️ 读取错误 {os.path.basename(f_path)}: {e}")

    if not all_durations:
        print(f"❌ 没有找到 {attack_name} 的数据。")
        return

    # 转换 numpy 数组加速计算
    arr_dur = np.array(all_durations)
    arr_pkt = np.array(all_pkts)
    arr_byte = np.array(all_bytes)

    total_count = len(arr_dur)

    print("\n" + "="*60)
    print(f"📊 {attack_name} 深度统计与截断建议")
    print(f"   样本总数: {total_count}")
    print("="*60)

    # ================= 定义统计函数 =================
    def print_stats(name, data, unit):
        print(f"\n🔹 {name} 分布:")
        p1 = np.percentile(data, 1)
        p25 = np.percentile(data, 25)
        p50 = np.percentile(data, 50)
        p90 = np.percentile(data, 90)
        p95 = np.percentile(data, 95)
        p99 = np.percentile(data, 99)
        p999 = np.percentile(data, 99.9) # 关注极端值
        max_val = np.max(data)

        print(f"   P1 (中位数): {p1:.2f} {unit}")
        print(f"   P25 (<UNK>): {p25:.2f} {unit}")
        print(f"   P50 (>UNK): {p50:.2f} {unit}")
        print(f"   P90        : {p90:.2f} {unit}")
        print(f"   P95        : {p95:.2f} {unit}")
        print(f"   P99        : {p99:.2f} {unit}")
        print(f"   P99.9      : {p999:.2f} {unit} <--- 关键截断参考点")
        print(f"   Max        : {max_val:.2f} {unit}")
        return p99, p999, max_val

    # ================= 1. Duration 分析 =================
    d99, d999, d_max = print_stats("Duration (持续时间)", arr_dur, "s")

    print(f"\n💡 TIMEOUT 设置建议:")
    if d99 < 15.0:
        print(f"   ✅ 保持 15s: 它可以覆盖 99% 以上的流量，非常安全。")
    else:
        print(f"   ⚠️ 建议调整: 15s 会切断部分长连接。如果这很重要，考虑设为 {d99:.0f}s (P99)。")

    # ================= 2. Packet Count 分析 =================
    p99, p999, p_max = print_stats("Packet Count (包数量)", arr_pkt, "pkts")

    # 寻找 MongoDB 安全阈值
    # 假设每个包存入 DB 大概占 200-300 Bytes (Header + Hex Payload)
    # 16MB (Mongo Limit) / 300 Bytes ≈ 53,000 包
    # 但为了安全，通常建议不超过 5000-10000

    print(f"\n💡 MAX_PACKET_COUNT 截断建议:")

    safe_threshold = 5000
    cutoff = int(p999) # 默认推荐 P99.9 作为截断点

    if p_max > safe_threshold:
        print(f"   🚨 发现巨型流 (Max: {p_max})！这会导致 MongoDB 报错。")
        print(f"   👉 强烈建议截断值: {safe_threshold} (或者 {min(cutoff, safe_threshold)})")
        print(f"   (这能保留 99.9% 的会话完整性，同时扔掉极端的异常值)")
    else:
        print(f"   ✅ 数据很小 (Max: {p_max})，即使设为 10000 也很安全。")

    # ================= 3. Byte Size 分析 =================
    # 这是原始流量大小，存入 MongoDB 后体积会膨胀 (因为 Hex 字符串是 2 倍，加上 JSON 结构)
    b99, b999, b_max = print_stats("Total Bytes (流量大小)", arr_byte, "Bytes")

    # 估算 MongoDB 文档大小 (粗略估计：原始字节 * 2.5 + Overhead)
    est_mongo_size_mb = (b_max * 2.5) / (1024 * 1024)

    print(f"\n💾 数据库文档大小估算:")
    print(f"   最大原始流量: {b_max / 1024 / 1024:.2f} MB")
    print(f"   估算 MongoDB 文档大小: ~{est_mongo_size_mb:.2f} MB")

    if est_mongo_size_mb > 16.0:
        print(f"   ❌ 红色警报: 最大的会话存入数据库后将超过 16MB 限制！")
        print(f"   必须使用 Packet Count 截断来限制大小。")
    else:
        print(f"   ✅ 绿色安全: 目前最大的会话也能塞进 MongoDB (上限 16MB)。")

# ================= 使用 =================
folder = r"C:\Users\zyf17\Desktop\NOT-IOT\processed"
suggest_truncation_thresholds(folder, 'scanning')

📂 正在扫描 23 个文件，分析攻击类型: 'scanning' ...


100%|██████████| 23/23 [00:21<00:00,  1.06file/s]



📊 scanning 深度统计与截断建议
   样本总数: 7140161

🔹 Duration (持续时间) 分布:
   P1 (中位数): 0.00 s
   P25 (<UNK>): 0.00 s
   P50 (>UNK): 0.00 s
   P90        : 0.00 s
   P95        : 0.02 s
   P99        : 3.52 s
   P99.9      : 10.04 s <--- 关键截断参考点
   Max        : 786.93 s

💡 TIMEOUT 设置建议:
   ✅ 保持 15s: 它可以覆盖 99% 以上的流量，非常安全。

🔹 Packet Count (包数量) 分布:
   P1 (中位数): 1.00 pkts
   P25 (<UNK>): 1.00 pkts
   P50 (>UNK): 1.00 pkts
   P90        : 2.00 pkts
   P95        : 2.00 pkts
   P99        : 9.00 pkts
   P99.9      : 17.00 pkts <--- 关键截断参考点
   Max        : 16745.00 pkts

💡 MAX_PACKET_COUNT 截断建议:
   🚨 发现巨型流 (Max: 16745)！这会导致 MongoDB 报错。
   👉 强烈建议截断值: 5000 (或者 17)
   (这能保留 99.9% 的会话完整性，同时扔掉极端的异常值)

🔹 Total Bytes (流量大小) 分布:
   P1 (中位数): 0.00 Bytes
   P25 (<UNK>): 0.00 Bytes
   P50 (>UNK): 0.00 Bytes
   P90        : 0.00 Bytes
   P95        : 0.00 Bytes
   P99        : 278.00 Bytes
   P99.9      : 2931.00 Bytes <--- 关键截断参考点
   Max        : 2039327532.00 Bytes

💾 数据库文档大小估算:
   最大原始流量: 1944.85 MB
   估算 MongoDB